# FlightAware's yesterday cancelled flights

Attempt to scrape FlightAware's list of cancelled flights: https://fr.flightaware.com/adsb/stats/


In [1]:
#%cd "C:\Users\ROSA_L\PycharmProjects\scraper\notebooks"
%cd ".."

C:\Users\ROSA_L\PycharmProjects\scraper


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from scraper.core.utils import logger
import logging
logger.level = logging.DEBUG

In [17]:
import pandas as pd

URL=r'https://flightaware.com/live/cancelled/yesterday'

#df = pd.read_html(URL, attrs={"class":"prettyTable sortable"})
df = pd.read_html(URL, attrs={"id":"airline_cancellation_board"})


ValueError: No tables found

In [88]:
df[0]

,By airline,Unnamed: 1,By origin airport,Unnamed: 3,By destination airport
0,By airline,NaN,By origin airport,NaN,By destination airport
1,Could not load airline statisticsPlease try ag...,NaN,Could not load origin statisticsPlease try aga...,NaN,Could not load destination statisticsPlease tr...


In [65]:
import requests
from IPython.core.display import display, HTML

#tables = pd.read_html(requests.get(url, 
#                                   headers={'User-agent': 'Mozilla/5.0'}).text, 
#                      attrs={"class":"tabelle_grafik"})[0]

response = requests.get(URL, headers={'ACCEPT': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                                      'ACCEPT-ENCODING': 'gzip,deflate',
                                      'ACCEPT-LANGUAGE': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
                                      'CONNECTION': 'keepalive',
                                      'USER-AGENT': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'})
page = display(HTML(response.text))
                        
#tables = pd.read_html(response.content, attrs={"class":"cancellation_boards"})

SSLError: HTTPSConnectionPool(host='flightaware.com', port=443): Max retries exceeded with url: /live/cancelled/yesterday (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))

## Conclusion for this first try

Dynamic page, with Javascript. We need *selenium*.


# Let's try with selenium

Requirement for using selenium for automating browser interaction:

install browser driver: https://sites.google.com/a/chromium.org/chromedriver/downloads

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pathlib import Path

cd_cmd = !echo %cd%
current_dir = Path(cd_cmd[0])

#CHROME_PATH = '/usr/bin/google-chrome'
CHROMEDRIVER_PATH = current_dir / r"drivers" / r"chromedriver.exe"
#WINDOW_SIZE = "1920,1080"

print(f"CHROMEDRIVER_PATH={CHROMEDRIVER_PATH}")

URL=r'https://flightaware.com/live/cancelled/yesterday'

chrome_options = Options()  
#chrome_options.add_argument("--headless")  
#chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
#chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=str(CHROMEDRIVER_PATH),
                          options=chrome_options
                         )  

#driver.get_screenshot_as_file("capture.png")
#driver.close()

CHROMEDRIVER_PATH=C:\Users\ROSA_L\PycharmProjects\scraper\drivers\chromedriver.exe


DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63542/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"extensions": [], "args": []}}}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:63542
DEBUG:urllib3.connectionpool:http://127.0.0.1:63542 "POST /session HTTP/1.1" 200 680
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request


## Time to read the section containing tables and save it to file

In [70]:
#from IPython.core.display import display, HTML
from pathlib import Path
from datetime import date, timedelta
import time
import pandas as pd

MAX_RETRY = 3
FILESTORE = Path('C:\\Users\\ROSA_L\\Downloads')
SLEEP = 5

retry = 0

filename = f"com_flightaware_{yesterday.strftime('%Y%m%d')}.html"
file = FILESTORE / filename 
    
while retry < MAX_RETRY:
    try:
        driver.get(URL)
        print(f"Sleeping {SLEEP} seconds to wait page download.")
        time.sleep(SLEEP)
        boards = driver.find_element_by_class_name("cancellation_boards")
        html = boards.get_attribute('innerHTML')
        
        # throws ValueError if no table found
        pd.read_html(html)
        
        file.write_text(html, encoding='UTF-8')
        print(f"Data written to {filename}")
        break
    except ValueError:
        retry += 1
        print(f"Attempt #{retry}: no table found!")
        
driver.close()

#file.read_text(encoding='UTF8')

DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63542/session/4413199ad9cbb929c2d2af19195455e7/url {"url": "https://flightaware.com/live/cancelled/yesterday"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63542 "POST /session/4413199ad9cbb929c2d2af19195455e7/url HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request


Sleeping 5 seconds to wait page download.


DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63542/session/4413199ad9cbb929c2d2af19195455e7/element {"using": "css selector", "value": ".cancellation_boards"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63542 "POST /session/4413199ad9cbb929c2d2af19195455e7/element HTTP/1.1" 200 88
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63542/session/4413199ad9cbb929c2d2af19195455e7/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.p

Data written to com_flightaware_20200323.html


In [118]:
import pandas as pd
from pathlib import Path
FILESTORE = Path('C:\\Users\\ROSA_L\\Downloads')
file = FILESTORE / "com_flightaware_20200323.html"

dfs = pd.read_html(file.read_text(encoding='UTF-8'))

#print(f"size: {len(dfs)}")

columns = ["cancelled_n", "cancelled_p", "delayed_n","delayed_p", "entity"]
category = ["airline", "airport", "airport"]
detail = ["airline", "origin_airport", "destination_airport"]


results = []

for i, df in enumerate(dfs):
    #print(i)
    df.columns = df.columns.droplevel()
    df.columns = columns
    df["category"] = category[i]
    df["detail"] = detail[i]
    #display(df.head())
    results.append(df)
    
df = pd.concat(results)
# flow: cancelled_flights, delayed_flights
# unit: unit, percent
df = df.melt(id_vars=["entity", "category", "detail"], var_name=["flow_unit"])
df["flow"] = df["flow_unit"].apply(lambda x: "".join([x.split("_")[0], "_flights"]).upper())
df["unit"] = df["flow_unit"].apply(lambda x: "IND" if x.split("_")[-1] == "n" else "PERC")
del df["flow_unit"]
display(df.head())

TypeError: Cannot read object of type 'WindowsPath'

In [128]:
from datetime import datetime

dt = "com_flightaware_20200323".split("_")[-1]

datetime.strptime(dt, "%Y%m%d")



datetime.datetime(2020, 3, 23, 0, 0)

## Let's read airlines and airports from entity dimension


In [45]:
endpoint = "http://vipenta:8000/dimension/entity"

import requests

response = requests.get(endpoint)
data = []
if response.ok:
   data = response.json()

In [120]:
import pandas as pd
airlines = pd.DataFrame([x for x in data if x['category'] == 'airline'])
airports = pd.DataFrame([x for x in data if x['category'] == 'airport'])
regions = pd.DataFrame([x for x in data if x['category'] == 'region'])
airlines['name'] = airlines['long_name'].str.split(' - ').str[-1]

In [113]:
display(airports.head())
display(airlines.head())

,category,code,date_created,date_modified,id,long_name,meta_data
0,airport,WKK,2020-03-20T22:11:53.447000+00:00,2020-03-20T22:11:53.447000+00:00,5424,US-AK - Aleknagik / New Airport,"{""iso_country"": ""US"", ""iso_region"": ""US-AK"", ""..."
1,airport,HIR,2020-03-20T22:11:53.450000+00:00,2020-03-20T22:11:53.450000+00:00,5425,SB-CT - Honiara International Airport,"{""continent"": ""OC"", ""iso_country"": ""SB"", ""iso_..."
2,airport,MUA,2020-03-20T22:11:53.453000+00:00,2020-03-20T22:11:53.453000+00:00,5426,SB-WE - Munda Airport,"{""continent"": ""OC"", ""iso_country"": ""SB"", ""iso_..."
3,airport,AHJ,2020-03-20T22:11:53.457000+00:00,2020-03-20T22:11:53.457000+00:00,5427,CN-51 - Hongyuan Airport,"{""continent"": ""AS"", ""iso_country"": ""CN"", ""iso_..."
4,airport,INU,2020-03-20T22:11:53.460000+00:00,2020-03-20T22:11:53.460000+00:00,5428,NR-14 - Nauru International Airport,"{""continent"": ""OC"", ""iso_country"": ""NR"", ""iso_..."


,category,code,date_created,date_modified,id,long_name,meta_data,name
0,airline,-,2020-03-21T02:44:22.257000+00:00,2020-03-21T02:44:22.257000+00:00,14629,- - Private flight,"{""iata_code"": ""-"", ""active"": ""Y""}",Private flight
1,airline,GNL,2020-03-21T02:44:22.263000+00:00,2020-03-21T02:44:22.263000+00:00,14630,GNL - 135 Airways,"{""icao_code"": ""GNL"", ""callsign"": ""GENERAL"", ""c...",135 Airways
2,airline,RNX,2020-03-21T02:44:22.267000+00:00,2020-03-21T02:44:22.267000+00:00,14631,RNX - 1Time Airline,"{""iata_code"": ""1T"", ""icao_code"": ""RNX"", ""calls...",1Time Airline
3,airline,WYT,2020-03-21T02:44:22.273000+00:00,2020-03-21T02:44:22.273000+00:00,14632,WYT - 2 Sqn No 1 Elementary Flying Training Sc...,"{""icao_code"": ""WYT"", ""country"": ""United Kingdo...",2 Sqn No 1 Elementary Flying Training School
4,airline,TFU,2020-03-21T02:44:22.277000+00:00,2020-03-21T02:44:22.277000+00:00,14633,TFU - 213 Flight Unit,"{""icao_code"": ""TFU"", ""country"": ""Russia"", ""act...",213 Flight Unit


## Let's join entities with tables

In [116]:
# df.loc[df['category'] == 'airport','entity_code'] = df['entity'].str.extract(pat=r'\(([A-Z]{3})\)')
df['entity_code'] = df['entity'].str.extract(pat=r'\(([A-Z]{3})\)')
#pd.merge(df, airlines[['name','code']], left_on='entity', right_on='name',how='left', indicator='join_result' )
df = df[df['category'] == 'airport']
df

In [151]:
from pandas.io.json import json_normalize
airports[['code', 'meta_data']]
df2 = json_normalize(airports['meta_data'].map(eval))
airp = pd.concat([airports, df2], axis=1)
display(airp[['code', 'iso_country']].sort_values(by='code'))
pd.merge(df, airp[['code', 'iso_country']], left_on='entity_code', right_on='code', how='left', indicator='mr')['mr']

,code,iso_country
2542,AAA,PF
1124,AAC,EG
403,AAE,DZ
692,AAL,DK
841,AAM,ZA
2726,AAN,AE
3455,AAO,VE
1201,AAP,ID
3668,AAQ,RU
676,AAR,DK


[both]
Categories (1, object): [both]

In [140]:
df2

,continent,iata_code,iso_country,iso_region,municipality,wikipedia_link
0,NaN,WKK,US,US-AK,Aleknagik,https://en.wikipedia.org/wiki/Aleknagik_Airport
1,OC,HIR,SB,SB-CT,Honiara,https://en.wikipedia.org/wiki/Honiara_Internat...
2,OC,MUA,SB,SB-WE,NaN,https://en.wikipedia.org/wiki/Munda_Airport
3,AS,AHJ,CN,CN-51,Aba,https://en.wikipedia.org/wiki/Hongyuan_Airport
4,OC,INU,NR,NR-14,Yaren District,https://en.wikipedia.org/wiki/Nauru_Internatio...
5,AS,AXF,CN,CN-15,Bayanhot,https://en.wikipedia.org/wiki/Alxa_Left_Banner...
6,OC,BUA,PG,PG-NSB,Buka Island,https://en.wikipedia.org/wiki/Buka_Airport
7,OC,CMU,PG,PG-CPK,Kundiawa,https://en.wikipedia.org/wiki/Chimbu_Airport
8,OC,DAU,PG,PG-WPD,Daru,https://en.wikipedia.org/wiki/Daru_Airport
9,OC,GKA,PG,PG-EHG,Goronka,https://en.wikipedia.org/wiki/Goroka_Airport


## Let's parse the tables

In [13]:
import pandas as pd
dfs = pd.read_html(file.read_text(encoding='UTF-8'))

for i, df in enumerate(dfs):
    print(i)
    #df.reset_index(inplace=True)
    df.reset_index(inplace=True, level=['None', '%', '#', '%', 'Unnamed: 4_level_1'])
    display(df.head())

0


KeyError: 'Level None must be same as name (None)'

In [166]:
print(dfs[0].columns.get_level_values(1))
#dfs[0]['Cancelled']['%']
df = dfs[0][['Unnamed: 4_level_0', 'Cancelled', 'Delayed','Airline',  'Unnamed: 3_level_0']]


Index(['#', '%', '#', '%', 'Unnamed: 4_level_1'], dtype='object')


In [7]:
df.reset_index(inplace=True, level=['#', '%', '#', '%', 'Unnamed: 4_level_1'])
df

KeyError: 'Level # must be same as name (None)'

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from scraper.core.utils import logger
import logging
logger = logging.getLogger()
logger.level = logging.DEBUG

In [22]:
import scraper.jobs.com_flightaware.job as job

f = job.CancelledFlightsJob(full_load=True)
#f = job.CancelledFlightsJob()

In [23]:
#f.run(download=True)
f.run(download=True)

DEBUG:scraper.jobs.com_flightaware.job:Generating sources...
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-27 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-26 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-25 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-24 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-23 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-22 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-21 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-20 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-19 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-18 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-17 00:00:00
DEBUG:scraper.jobs.com_flightaware.job:Creating source for 2020-03-16 

In [17]:
#f.insert_new_dynamic_dim()
#f.upsert()
f.dynamic_dim

defaultdict(list, {'source': [], 'provider': [], 'detail': []})

In [18]:
f.sources

['BaseSource(c...0200326.html)',
 'BaseSource(c...0200325.html)',
 'BaseSource(c...0200324.html)',
 'BaseSource(c...0200323.html)',
 'BaseSource(c...0200322.html)',
 'BaseSource(c...0200321.html)',
 'BaseSource(c...0200320.html)',
 'BaseSource(c...0200319.html)',
 'BaseSource(c...0200318.html)',
 'BaseSource(c...0200317.html)',
 'BaseSource(c...0200316.html)',
 'BaseSource(c...0200315.html)',
 'BaseSource(c...0200314.html)',
 'BaseSource(c...0200313.html)',
 'BaseSource(c...0200312.html)',
 'BaseSource(c...0200311.html)',
 'BaseSource(c...0200310.html)',
 'BaseSource(c...0200309.html)']

In [25]:
[vars(source) for source in f.sources]

[{'code': 'com_flightaware_20200325',
  'url': 'https://flightaware.com/live/cancelled/yesterday',
  'path': 'com_flightaware_20200325.html',
  'long_name': 'COM_FLIGHTAWARE cancelled flights 20200325',
  'checksum': '1e4efdbc7f382aa2fdfff93bd4e0a209'},
 {'code': 'com_flightaware_20200324',
  'url': 'https://flightaware.com/live/cancelled/yesterday',
  'path': 'com_flightaware_20200324.html',
  'long_name': 'COM_FLIGHTAWARE cancelled flights 20200324',
  'checksum': 'c94da8088cbc4be9c9588ac0724474f6'},
 {'code': 'com_flightaware_20200323',
  'url': 'https://flightaware.com/live/cancelled/yesterday',
  'path': 'com_flightaware_20200323.html',
  'long_name': 'COM_FLIGHTAWARE cancelled flights 20200323',
  'checksum': 'eafd60c193f13d7cabafd1341b9a1fbe'},
 {'code': 'com_flightaware_20200322',
  'url': 'https://flightaware.com/live/cancelled/yesterday',
  'path': 'com_flightaware_20200322.html',
  'long_name': 'COM_FLIGHTAWARE cancelled flights 20200322',
  'checksum': 'f26aa2fabd07a698a578

In [73]:
import pandas as pd
df_test = pd.DataFrame(f.data)

In [76]:
df_test[df_test['source']  == 'com_flightaware_20200314']

,area,detail,entity,flow,frequency,original,period,product,provider,source,to_area,unit,value
800,ITALY,COM_FLIGHTAWARE_OA,FCO,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,104.00
801,GERMANY,COM_FLIGHTAWARE_OA,FRA,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,79.00
802,CHINA,COM_FLIGHTAWARE_OA,PEK,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,69.00
803,FRANCE,COM_FLIGHTAWARE_OA,CDG,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,66.00
804,CHINA,COM_FLIGHTAWARE_OA,SZX,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,64.00
805,CHINA,COM_FLIGHTAWARE_OA,CAN,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,59.00
806,ITALY,COM_FLIGHTAWARE_OA,MXP,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,58.00
807,CHINA,COM_FLIGHTAWARE_OA,CTU,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,58.00
808,PHILIPPINE,COM_FLIGHTAWARE_OA,MNL,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,55.00
809,NETHLAND,COM_FLIGHTAWARE_OA,AMS,CANCELLED_FLIGHTS,Daily,True,2020-03-14,None,COM_FLIGHTAWARE,com_flightaware_20200314,None,IND,53.00


In [54]:
df_test[df_test['entity'] == 'ATL']
df_test['value2'] = df_test.apply(lambda x: x['value'].str[:-1] if x['unit'] == 'PERC' else x['value'])

KeyError: 'entity'

In [14]:
f.download_source(f.sources[0])

In [8]:
import pandas as pd
from datetime import date
for period in pd.date_range(date.today(), date.today()):
    print(period)

2020-03-13 00:00:00


In [3]:
driver.close()

In [3]:
from scraper.settings import FILE_STORE_PATH
import pandas as pd

path = FILE_STORE_PATH / 'com_ourairports_airports.csv'
df = pd.read_csv(path)
df = df[((df['type'] == 'large_airport') | (df['type'] == 'medium_airport')) & ~df['iata_code'].isna()]
df

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
6207,12243,5A8,medium_airport,Aleknagik / New Airport,59.282600,-158.617996,66.0,NaN,US,US-AK,Aleknagik,yes,5A8,WKK,5A8,NaN,https://en.wikipedia.org/wiki/Aleknagik_Airport,NaN
10468,3,AGGH,medium_airport,Honiara International Airport,-9.428000,160.054993,28.0,OC,SB,SB-CT,Honiara,yes,AGGH,HIR,NaN,NaN,https://en.wikipedia.org/wiki/Honiara_Internat...,Henderson Field
10473,4,AGGM,medium_airport,Munda Airport,-8.327970,157.263000,10.0,OC,SB,SB-WE,NaN,yes,AGGM,MUA,NaN,NaN,https://en.wikipedia.org/wiki/Munda_Airport,NaN
10495,312371,AHJ,medium_airport,Hongyuan Airport,32.531540,102.352240,11600.0,AS,CN,CN-51,Aba,no,ZUHY,AHJ,NaN,NaN,https://en.wikipedia.org/wiki/Hongyuan_Airport,NaN
10719,8,ANYN,medium_airport,Nauru International Airport,-0.547458,166.919006,22.0,OC,NR,NR-14,Yaren District,yes,ANYN,INU,NaN,NaN,https://en.wikipedia.org/wiki/Nauru_Internatio...,ANAU
11681,314558,AXF,medium_airport,Alxa Left Banner Bayanhot Airport,38.748310,105.588580,4560.0,AS,CN,CN-15,Bayanhot,yes,ZBAL,AXF,NaN,NaN,https://en.wikipedia.org/wiki/Alxa_Left_Banner...,NaN
11685,53,AYBK,medium_airport,Buka Airport,-5.422320,154.673004,11.0,OC,PG,PG-NSB,Buka Island,yes,AYBK,BUA,NaN,NaN,https://en.wikipedia.org/wiki/Buka_Airport,NaN
11692,54,AYCH,medium_airport,Chimbu Airport,-6.024290,144.970993,4974.0,OC,PG,PG-CPK,Kundiawa,yes,AYCH,CMU,NaN,NaN,https://en.wikipedia.org/wiki/Chimbu_Airport,NaN
11701,55,AYDU,medium_airport,Daru Airport,-9.086760,143.207993,20.0,OC,PG,PG-WPD,Daru,yes,AYDU,DAU,NaN,NaN,https://en.wikipedia.org/wiki/Daru_Airport,NaN
11724,56,AYGA,medium_airport,Goroka Airport,-6.081690,145.391998,5282.0,OC,PG,PG-EHG,Goronka,yes,AYGA,GKA,NaN,NaN,https://en.wikipedia.org/wiki/Goroka_Airport,NaN


In [63]:
from scraper.jobs.com_ourairports.job import OurAirportsJob
import logging

logger = logging.getLogger()

logger.level = logging.DEBUG

job = OurAirportsJob()
job.run()

INFO:scraper.jobs.com_ourairports.job:Getting sources...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 712570
DEBUG:scraper.core.job:download: True, parallel download: True
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ourairports.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ourairports.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ourairports.com:443
DEBUG:urllib3.connectionpool:https://ourairports.com:443 "GET /data/airports.csv HTTP/1.1" 200 8448345
DEBUG:urllib3.connectionpool:https://ourairports.com:443 "GET /data/countries.csv HTTP/1.1" 200 20648
DEBUG:urllib3.connectionpool:https://ourairports.com:443 "GET /data/regions.csv HTTP/1.1" 200 364582
INFO:scraper.core.utils:download_and_get_checksum: 6722.982168197632 ms
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG

In [22]:
job.sources

['BaseSource(c...airports.csv)',
 'BaseSource(c...ountries.csv)',
 'BaseSource(c..._regions.csv)']

In [17]:
[x for x in job.dynamic_dim['entity'] if x['long_name'] is None][:5]

[]

In [64]:
import pandas as pd
df2 = pd.DataFrame(job.dynamic_dim['entity'])

In [65]:
df2

""


In [7]:
df2.loc[df2['code'] == 'NA-U-A', 'long_name'] = df2['code'] + " (unassigned)"
df2.loc[df2['long_name'].isnull(), 'long_name'] = df2['code'] + " (unassigned)"

In [15]:
df2[df2['long_name'].isnull()] 

,category,code,long_name,meta_data
26,region,NA-CA,NaN,"{'local_code': 'CA', 'continent': 'AF', 'wikip..."
27,region,NA-ER,NaN,"{'local_code': 'ER', 'continent': 'AF', 'wikip..."
28,region,NA-HA,NaN,"{'local_code': 'HA', 'continent': 'AF', 'wikip..."
29,region,NA-KA,NaN,"{'local_code': 'KA', 'continent': 'AF', 'wikip..."
30,region,NA-KE,NaN,"{'local_code': 'KE', 'continent': 'AF', 'wikip..."
31,region,NA-KH,NaN,"{'local_code': 'KH', 'continent': 'AF', 'wikip..."
32,region,NA-KU,NaN,"{'local_code': 'KU', 'continent': 'AF', 'wikip..."
33,region,NA-KW,NaN,"{'local_code': 'KW', 'continent': 'AF', 'wikip..."
34,region,NA-OD,NaN,"{'local_code': 'OD', 'continent': 'AF', 'wikip..."
35,region,NA-OH,NaN,"{'local_code': 'OH', 'continent': 'AF', 'wikip..."


In [8]:
df2[df2['long_name'].isnull()]

,category,code,long_name,meta_data


In [127]:
[x for x in df2[df2['code'] == 'NA-CA']['meta_data'].items()]

[(26,
  {'local_code': 'CA',
   'continent': 'AF',
   'wikipedia_link': 'https://en.wikipedia.org/wiki/Zambezi_Region'})]

In [130]:
'https://en.wikipedia.org/wiki/Zambezi_Region'.split('/')[-1].split('_')[0]

'Zambezi'

In [105]:
df2[df2['long_name'].isnull()]

,category,code,long_name,meta_data


In [103]:
df2.loc[df2['long_name'].isnull(), 'long_name'] = 'test'

In [109]:
df2['long_name'] = df2.apply(lambda x:
                                   x['long_name'] if x['long_name'].notnull().all()
                                   else x['code'] + " (unassigned)", axis=1)

AttributeError: ("'str' object has no attribute 'notnull'", 'occurred at index 0')

In [113]:
df2.loc[df2['code'] == 'NA-U-A', 'long_name'] = df2['code'] + ' (unassigned)'

In [115]:
df2.loc[df2['long_name'] == 'test', 'long_name'] = df2['meta_data']

In [5]:
from scraper.jobs.org_openflights.job import OpenFlightsJob

flights = OpenFlightsJob()
flights.run()

In [70]:
flights.dynamic_dim['entity'][:5]

[{'code': 'NJS',
  'long_name': 'NJS - National Jet Systems',
  'category': 'airline',
  'meta_data': {'iata_code': 'NC',
   'icao_code': 'NJS',
   'callsign': 'NATIONAL JET',
   'country': 'Australia',
   'active': 'Y'}},
 {'code': 'NTW',
  'long_name': 'NTW - Nationwide Airlines',
  'category': 'airline',
  'meta_data': {'iata_code': 'CE',
   'icao_code': 'NTW',
   'callsign': 'NATIONWIDE',
   'country': 'South Africa',
   'active': 'Y'}},
 {'code': 'RON',
  'long_name': 'RON - Nauru Air Corporation',
  'category': 'airline',
  'meta_data': {'iata_code': 'ON',
   'icao_code': 'RON',
   'callsign': 'AIR NAURU',
   'country': 'Nauru',
   'active': 'Y'}},
 {'code': 'NEA',
  'long_name': 'NEA - New England Airlines',
  'category': 'airline',
  'meta_data': {'iata_code': 'EJ',
   'icao_code': 'NEA',
   'callsign': 'NEW ENGLAND',
   'country': 'United States',
   'active': 'Y'}},
 {'code': 'NTJ',
  'long_name': 'NTJ - NextJet',
  'category': 'airline',
  'meta_data': {'iata_code': '2N',
  

In [81]:
df = pd.DataFrame(flights.dynamic_dim['entity'])

In [83]:
df[df.long_name.isnull()]


,category,code,long_name,meta_data
68,airline,NaN,NaN,"{'country': 'Iraq', 'active': 'Y'}"
